In [7]:
import pandas as pd

In [8]:
import numpy as np

In [ ]:
from scipy.stats import chi2_contingency, chi2
from statsmodels.stats.multitest import multipletests

def combine_categories_if_needed(contingency_table, min_expected=5):
    """Combine columns of contingency table if any expected value is less than the minimum."""
    # Remove columns with all zero values (create a new copy to avoid SettingWithCopyWarning)
    contingency_table = contingency_table.loc[:, contingency_table.sum(axis=0) > 0].copy()

    while True:
        # Perform Chi-Square Test
        chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

        # Check if all expected values are >= min_expected
        if np.min(expected) >= min_expected:
            return contingency_table, chi2_stat, p_val, expected

        # Find the two adjacent columns with the smallest expected values and combine them
        col_to_combine = np.argmin(np.min(expected, axis=0))
        if col_to_combine == contingency_table.shape[1] - 1:
            col_to_combine -= 1  # Combine with the previous column if it's the last column

        # Combine the two columns (ensure operations create new copies)
        contingency_table.iloc[:, col_to_combine] += contingency_table.iloc[:, col_to_combine + 1]
        contingency_table = contingency_table.drop(contingency_table.columns[col_to_combine + 1], axis=1).copy()

def chi_square_test_for_trend(data, bonferroni_correction=True):
    """Perform Chi-Square Test for Trend for each pair of cell classes."""
    trend_results = []
    for class_1 in data.index:
        for class_2 in data.index:
            if class_1 >= class_2:
                continue

            # Subset data for the two classes
            subset = data.loc[[class_1, class_2]]

            # Assign scores to columns (ordinal categories)
            scores = np.arange(1, subset.shape[1] + 1)

            # Perform test for trend
            observed = subset.values
            expected = observed.sum(axis=0) * observed.sum(axis=1).reshape(-1, 1) / observed.sum()
            chi2_stat = np.sum(((observed - expected)**2 / expected) * scores)
            p_val = 1 - chi2.cdf(chi2_stat, df=1)

            trend_results.append((class_1, class_2, chi2_stat, p_val))

    # Apply Bonferroni correction
    if bonferroni_correction:
        p_vals = [res[3] for res in trend_results]
        _, corrected_p_vals, _, _ = multipletests(p_vals, method='bonferroni')
        for i, res in enumerate(trend_results):
            trend_results[i] = (*res[:3], corrected_p_vals[i])

    return trend_results

# Example data
columns = ['1-2', '3-4', '5-6', '7-8', '9-10', '11-12']
data = {
    'No Drd': [37, 4, 0, 0, 0, 0],
    'Drd1': [184, 32, 5, 1, 0, 0],
    'Drd2': [160, 57, 9, 0, 0, 0],
    'D1+D2': [150, 76, 27, 8, 3, 0],
}
contingency_table = pd.DataFrame(data, index=columns).T

# Step 1: Chi-Square Test of Independence
print("Performing Chi-Square Test of Independence...")
contingency_table, chi2_stat, p_val, expected = combine_categories_if_needed(contingency_table)
p_val_formatted = f"{p_val:.2e}" if p_val < 1e-4 else f"{p_val:.4f}"
print(f"Chi-Square Statistic: {chi2_stat:.4f}, p-value: {p_val_formatted}")

# Step 2: If significant, perform Chi-Square Test for Trend
if p_val < 0.05:
    print("Chi-Square Test significant. Performing post hoc Chi-Square Tests for Trend...")

    # Calculate number of comparisons and Bonferroni correction value
    num_comparisons = len(contingency_table.index) * (len(contingency_table.index) - 1) // 2
    bonferroni_alpha = 0.05 / num_comparisons
    print(f"Bonferroni-corrected significance level: {bonferroni_alpha:.4f}")

    trend_results = chi_square_test_for_trend(contingency_table)
    print("Post Hoc Chi-Square Tests for Trend (with Bonferroni correction):")
    for class_1, class_2, chi2_stat, p_val in trend_results:
        significance = "Significant" if p_val < bonferroni_alpha else "Not Significant"
        p_val_formatted = f"{p_val:.2e}" if p_val < 1e-4 else f"{p_val:.4f}"
        print(f"{class_1} vs {class_2}: Chi2 = {chi2_stat:.4f}, p-value = {p_val_formatted} ({significance})")
else:
    print("Chi-Square Test not significant. No further tests performed.")
